article : https://arxiv.org/pdf/1307.0048.pdf

L'idée du projet est d'implementer un algorithme mapreduce sur la régression linéaire pénalisée, quand $X \in \mathbb{R}^{n \times p}$ avec $p << n$.

Cela correspond à un type de problème ou le nombre de features $p$ (les caractéristiques d'un individu ou un produit) est assez petit et il est envisageable de les stocker en mémoire, alors que la taille du dataset $n$ et très grande et on voudrait faire du calcul distributé dessus

L'idée de l'agorithme est alors d'exprimer la quantité à minimiser en fonction des matrices ou des vecteurs dont la dimension est une fonction de $p$ ($p\times p$ ou $p$ en fait). Et calculer ces quantitées à partir de $X \in \mathbb{R}^{n \times p}$ en faisant une reduction sur $n$ (qui est la taille de notre dataset)

In [2]:
import numpy as np
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz

# Algorithm 1

En notant $X_c \in \mathbb{R}^{n \times p }$ la matrice centrée réduite de $X$. On a :
$$X_c = (X - \mathbb{1} (\bar{X_1}, \dots, \bar{X_p}))D^{-1}$$

et
$$\begin{align}
& & \ ||Y - \alpha \mathbb{1} - X \beta||_2^2 + p_\lambda(\beta) \\
= & \ ||Y - \alpha \mathbb{1} - (X_cD + \mathbb{1} (\bar{X_1}, \dots, \bar{X_p})) \beta||_2 + p_\lambda(\beta) \\
= & \ ||Y - (\alpha + (\bar{X_1}, \dots, \bar{X_p})) \beta) \mathbb{1} - X_c D \beta ||_2 + p_\lambda(\beta)
\end{align}$$

On a donc que minimiser:
$$||Y - \alpha \mathbb{1} - X \beta||_2^2 + p_\lambda(\beta) $$
revient à minimiser:

$$\begin{align}
||Y - \hat{\alpha} \mathbb{1} - X_c \hat{\beta}||_2^2 + p_\lambda(\hat{\beta})
\end{align}$$
Avec le changement de variable:

$$\begin{align} 
\hat{\alpha}&= \alpha + \left(\bar{X_1}, \dots, \bar{X_p}\right) \beta \\
\hat{\beta} &= D \beta
\end{align}
 $$
 
 avec D la matrice diagonale des déviations standards.
 
 Si on developpe la dernière équation on trouve:
 $$\begin{align}
\ &||Y - \hat{\alpha} \mathbb{1} - X_c \hat{\beta}||_2^2 + p_\lambda(\hat{\beta}) \\
= \ & Y^TY  + n \alpha^2 + \hat{\beta}^TX_c^TX_c\beta - 2 n \alpha \bar{Y} + 2 \alpha \mathbb{1}^X_c\beta - 2Y^TX_c\hat{\beta} + p_\lambda(\hat{\beta})\\
\end{align}$$
 
 Comme $X_c$ est centrée  le minimum est atteint pour $\hat{\alpha} = \bar{Y}$ et $\alpha \mathbb{1}^X_c = 0$.
 
 L'expression à minimiser devient donc:
 
 $$\begin{align}
  &\ \hat{\beta} = \arg \min_{\beta}  Y^TY - n \bar{Y}^2 + \beta^TX_c^TX_c\beta - 2Y^TX_c\beta + p_\lambda(\beta) \\
   \Leftrightarrow &\ \hat{\beta} =  \arg \min_{\beta} \beta^TX_c^TX_c\beta - 2Y^TX_c\beta + p_\lambda(\beta) \\
   \Leftrightarrow &\ \hat{\beta} =  \arg \min_{\beta} \beta^TD^{-1}(X^TX - n(\bar{X_1}, \dots, \bar{X_p})^T (\bar{X_1}, \dots, \bar{X_p}))D^{-1}\beta - 2(Y^TX - n \bar{Y}(\bar{X_1}, \dots, \bar{X_p}))D^{-1}\beta + p_\lambda(\beta)
 \end{align}
 $$
 
 $X^TX, Y^TX, D^{-1}, (\bar{X_1}, \dots, \bar{X_p})^T (\bar{X_1}, \dots, \bar{X_p})$ sont des matrices de taille $p \times p$. On peut par hypothèse les stocker en mémoire et résoudre ce problème par une des méthodes d'optimisation classiques (coordinate descent par exemple).
 
Lors de l'étape map-reduce on calculera $X^TX, Y^TX, \bar{Y}, (\bar{X_1}, \dots, \bar{X_p}), COV(X)$ à partir desquelles on retrouvera les quantités citées précédemment.


# Rajouter le fait qu'on fait de la cross validation en même temps (en faisant un reducebyKey)

In [3]:
from scipy.optimize import minimize 
def PenalizedLR_MR(Data, k, lambdas, penalizer="ridge"):
    """
    Data: an RDD each rows of which is a tuple (x, y)
    k: number of partitions for splitting (k>2 for implementation reasons)
    lambdas: list of lambdas to test on
    penalizer: penalization term (only "ridge" is avaialble for now)
    
    """
    def map_statistics(row):
        # calculate statistics for one row [size, mean(x), mean(y), Y^TY, y * x, cov(x)]
        x = row[0]
        y = row[1]
        return (np.random.randint(k), [1, x, y, y**2, y * x, np.zeros((len(row[0]), len(row[0])))])


    def reduce_statistics(row1, row2):
        #combined with map_statistics returns [size, mean(X), mean(Y), Y^TY, Y^TX, Cov(X)]
        s_1 = row1[0]
        s_2 = row2[0]
        mean_x = s_1 / (s_1 + s_2) * row1[1] + s_2 / (s_1 + s_2) * row2[1]
        mean_y = s_1 / (s_1 + s_2) * row1[2] + s_2 / (s_1 + s_2) * row2[2]
        
        mean_substraction = (row1[1] - row2[1]).reshape((1, -1))
        cov = s_1 / (s_1 + s_2) * row1[5] + s_2 / (s_1 + s_2) * row2[5] + s_1 * s_2 / (
            s_1 + s_2)**2 * (mean_substraction).T.dot(mean_substraction)
        emit = [s_1 + s_2, mean_x, mean_y, row1[3] +
                row2[3], row1[4] + row2[4], cov]
        return emit

    statistics = Data.map(map_statistics)
    statistics = statistics.reduceByKey(reduce_statistics)

    # Cross validation
    test_errors = []
    for lmbda in lambdas:
        error = 0
        for i in range(k):
            #do the split
            statistics_train = statistics.filter(lambda row: row[0] != i)
            
            statistics_test = statistics.filter(lambda row: row[0] == i).collect()[0][1]
            #calculate the needed dot products for ou train dataset
            size_train, means_X_train, mean_Y_train, YT_Y_train, YT_X_train, COV_X_train  = statistics_train.map(lambda row: row[1]).reduce(reduce_statistics)
            #reshape our vectors
            means_X_train = means_X_train.reshape(-1, 1)
            YT_X_train = YT_X_train.reshape(-1, 1)

            p = COV_X_train.shape[0]
            XT_X_train = size_train * (COV_X_train + means_X_train.dot(means_X_train.T))
            D_inv_train = np.diag([1 / np.sqrt(COV_X_train[i,i]) for i in range(p)])
            D_train = np.diag([np.sqrt(COV_X_train[i,i]) for i in range(p)])
            
            #calculate statistics foro our test dataset:
            size_test = statistics_test[0]
            means_X_test = statistics_test[1].reshape(-1, 1)
            mean_Y_test = statistics_test[2]
            YT_Y_test = statistics_test[3]
            YT_X_test = statistics_test[4].reshape(-1, 1)
            COV_X_test = statistics_test[5]
            
            
            XT_X_test = size_test * (COV_X_test + means_X_test.T.dot(means_X_test))
            D_inv_test = np.diag([1 / np.sqrt(COV_X_test[i,i]) for i in range(p)])
            D_test = np.diag([np.sqrt(COV_X_train[i,i]) for i in range(p)])
            
            def beta_objective(beta):
                pen_term = 0
                if penalizer == "Ridge":
                    pen_term = lmbda + np.linalg.norm(beta**2)
                #the simplified objective function for beta
                linear_term = -(YT_X_train - size_train * mean_Y_train * means_X_train).T.dot(D_inv_train).dot(beta)
                quadratic_term = 1 /2 * beta.dot(D_inv_train).dot(XT_X_train - size_train * means_X_train.dot(means_X_train.T)).dot(D_inv_train).dot(beta)
                return linear_term + quadratic_term + pen_term
            
            
            alpha_hat = mean_Y_test
            beta_hat = minimize(beta_objective, np.zeros(p), method="CG").x
            
            def test_error(alpha, beta):
                quadratic_term = YT_Y_test - size_test * alpha**2 + beta.dot(D_inv_test).dot(XT_X_test - size_test * means_X_test.T.dot(means_X_test)).dot(D_inv_test).dot(beta)
                double_term = -2 * alpha * mean_Y_test -2 * (YT_X_test - size_test * mean_Y_test * means_X_test).T.dot(D_inv_test).dot(beta)
                return quadratic_term + double_term
            error += test_error(alpha_hat, beta_hat)
        test_errors.append(error)
    
    #choose best lambda:
    best_i = np.argmin(test_errors)
    best_lambda = lambdas[best_i]

    #calculate the needed dot products
    size, means_X, mean_Y, YT_Y, YT_X, COV_X  = statistics.map(lambda x : x[1]).reduce(reduce_statistics)
    
    #reshape our vectors
    means_X = means_X.reshape(-1, 1)
    YT_X = YT_X.reshape(-1, 1)
    
    p = COV_X.shape[0]
    XT_X = size * (COV_X + means_X.dot(means_X.T))
    D_inv = np.diag([1 / np.sqrt(COV_X[i,i]) for i in range(p)])
    D = np.diag([np.sqrt(COV_X[i,i]) for i in range(p)])
    
    def beta_objective(beta):
        pen_term = 0
        if penalizer == "Ridge":
            pen_term = lmbda + np.linalg.norm(beta**2)
        #changing variables
        beta = D.dot(beta)
        #the simplified objective function for beta
        linear_term = -(YT_X - size * mean_Y * means_X).T.dot(D_inv).dot(beta)
        quadratic_term = 1 /2 * beta.dot(D_inv).dot(XT_X - size * means_X.dot(means_X.T)).dot(D_inv).dot(beta)
        return linear_term + quadratic_term + pen_term
    
    #calculate final alpha, beta
    beta = minimize(beta_objective, np.zeros(p), method="CG").x
    alpha_hat = mean_Y
    alpha = alpha_hat - means_X.T.dot(beta)
    
    return (alpha, beta, best_lambda)

# Quelques tests

n = 500

In [4]:
p = 100
n = 500
cov = toeplitz(0.5 ** np.arange(p))
X = multivariate_normal(np.zeros(p), np.eye(p), n)


idx = np.arange(p)
coefs = (idx % 2) * np.exp(-idx / 10.)
coefs[40:] = 0.
    
y = X.dot(coefs)

Data_array = [(X[i], y[i]) for i in range(n)]

Data_rdd = sc.parallelize(Data_array)


In [5]:
alpha, beta, lmbda = PenalizedLR_MR(Data=Data_rdd, k=5, lambdas=np.logspace(-2, 2, 5), penalizer="Ridge")

print(alpha, np.linalg.norm(beta - coefs, ord=2), lmbda)

[4.81162156e-05] 0.004597685090530298 0.01


In [7]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
scores = []
lambdas = np.logspace(-2, 2, 5)
for lmbda in lambdas:
    clf = Ridge(lmbda, fit_intercept=True)
    score = cross_val_score(clf, X, y, cv=5).mean()
    scores.append(score)

best_i = np.argmin(scores)
best_lambda = lambdas[best_i]
clf = Ridge(best_lambda, fit_intercept=True)
clf.fit(X, y)
print(clf.intercept_, np.linalg.norm(clf.coef_ - coefs, ord=2))

0.008422742420145918 0.32487655733699944


In [5]:
%timeit PenalizedLR_MR(Data=Data_rdd, k=5, lambdas=np.logspace(-2, 2, 5), penalizer="Ridge")

1 loop, best of 3: 14.3 s per loop


In [8]:
%%timeit
scores = []
lambdas = np.logspace(-2, 2, 5)
for lmbda in lambdas:
    clf = Ridge(lmbda, fit_intercept=True)
    score = cross_val_score(clf, X, y, cv=5).mean()
    scores.append(score)

best_i = np.argmin(scores)
best_lambda = lambdas[best_i]
clf = Ridge(best_lambda, fit_intercept=True)
clf.fit(X, y)

10 loops, best of 3: 63.8 ms per loop


$n$ = 10000

In [9]:
p = 100
n = 10000
cov = toeplitz(0.5 ** np.arange(p))
X = multivariate_normal(np.zeros(p), np.eye(p), n)


idx = np.arange(p)
coefs = (idx % 2) * np.exp(-idx / 10.)
coefs[40:] = 0.
    
y = X.dot(coefs)

Data_array = [(X[i], y[i]) for i in range(n)]

Data_rdd = sc.parallelize(Data_array)

In [10]:
%timeit PenalizedLR_MR(Data=Data_rdd, k=5, lambdas=np.logspace(-2, 2, 5), penalizer="Ridge")

1 loop, best of 3: 20.3 s per loop


In [11]:
%%timeit
scores = []
lambdas = np.logspace(-2, 2, 5)
for lmbda in lambdas:
    clf = Ridge(lmbda, fit_intercept=True)
    score = cross_val_score(clf, X, y, cv=5).mean()
    scores.append(score)

best_i = np.argmin(scores)
best_lambda = lambdas[best_i]
clf = Ridge(best_lambda, fit_intercept=True)
clf.fit(X, y)

1 loop, best of 3: 603 ms per loop


$n$ = 100000

In [12]:
p = 100
n = 100000
cov = toeplitz(0.5 ** np.arange(p))
X = multivariate_normal(np.zeros(p), np.eye(p), n)


idx = np.arange(p)
coefs = (idx % 2) * np.exp(-idx / 10.)
coefs[40:] = 0.
    
y = X.dot(coefs)

Data_array = [(X[i], y[i]) for i in range(n)]

Data_rdd = sc.parallelize(Data_array)

In [13]:
%timeit PenalizedLR_MR(Data=Data_rdd, k=5, lambdas=np.logspace(-2, 2, 5), penalizer="Ridge")

1 loop, best of 3: 24.2 s per loop


In [14]:
%%timeit
scores = []
lambdas = np.logspace(-2, 2, 5)
for lmbda in lambdas:
    clf = Ridge(lmbda, fit_intercept=True)
    score = cross_val_score(clf, X, y, cv=5).mean()
    scores.append(score)

best_i = np.argmin(scores)
best_lambda = lambdas[best_i]
clf = Ridge(best_lambda, fit_intercept=True)
clf.fit(X, y)

1 loop, best of 3: 5.54 s per loop


On remarque que pour $n$ petit, la regression Ridge de scikit-leran est beaucoup plus rapide que notre implémentation, ce qui est normal vu qu'elle est spécialement optimisée pour python (probablement codée en cython même).

Cependant quand $n$ augmente l'écart diminue sensiblement passant de au moins 20 fois pour $n=500$ à moins de 5 fois pour $n=100000$. Ce qui laisse supposer un certain avantage de notre méthode si elle était executée sur un grand nombre de machine et son optimisation serait plus optimisée